In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with RAGAS & Vertex AI Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag-evaluation/ragas_with_gemini.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fretrieval-augmented-generation%2Frag-evaluation%2Fragas_with_gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag-evaluation/ragas_with_gemini.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/rag-evaluation/ragas_with_gemini.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Aditya Rane](https://github.com/Adi8885) |

## Overview

### [RAGAS](https://docs.ragas.io/en/stable/)

Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines. RAG denotes a class of LLM applications that use external data to augment the LLM's context. There are existing tools and frameworks that help you build these pipelines but evaluating it and quantifying your pipeline performance can be hard. This is where Ragas (RAG Assessment) comes in.

### Gemini

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini models.

### Vertex AI Gemini API

The Vertex AI Gemini API provides a unified interface for interacting with Gemini models.

You can interact with the Gemini API using the following methods:

- Use the [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) for quick testing and command generation
- Use cURL commands
- Use the Vertex AI SDK

This notebook focuses on using the **Gemini model with RAGAS**

For more information, see the [Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) documentation.

### Objectives

In this notebook we will focus on using the Vertex AI Gemini API with RAGAS
We will use the Gemini Pro (`gemini-1.5-pro`) model for Q&A evaluation.

You will complete the following tasks:

- Install the Vertex AI SDK for Python
- Use the Vertex AI Gemini API to interact with each model
  - Gemini Pro (`gemini-1.5-pro`) model:
    - Q&A Generation
    - Evaluate Q&A performance with RAGAS

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK for Python

In [ ]:
! pip3 install --user install ragas==0.1.6 \
datasets==2.18.0 \
langchain==0.1.14 \
langchain-google-vertexai==1.0.5 \
langchain-chroma==0.1.1 \
chromadb==0.5.0 \
pypdf==4.2.0 \

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Run the following cell to restart the current kernel.

The restart process might take a minute or so.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

After the restart is complete, continue to the next step.

<div class="alert alert-block alert-warning">
<b>⚠️ Wait for the kernel to finish restarting before you continue. ⚠️</b>
</div>

## Import libraries

In [ ]:
import pandas as pd
import vertexai
from datasets import Dataset
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings
from ragas import evaluate

# Important to make Gemini Work with RAGAS
from ragas.llms.base import LangchainLLMWrapper
from ragas.metrics import (
    answer_relevancy,
    answer_similarity,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

In [ ]:
# TODO(developer): Update the below lines
PROJECT_ID = "<your_project>"
LOCATION = "<your_region>"

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Use Vertex AI models

The [Gemini-1.5-pro](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/overview) models are designed to handle multimodal inputs.

In [ ]:
# Load the Gemini Pro model
llm = VertexAI(model_name="gemini-1.5-pro")

The [Vertex AI Embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings) models are designed to convert text to dense vector representations

In [ ]:
# Load Embeddings Models
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003")

## Create a local Vector DB
### Load the document

In [ ]:
# source document
document_uri = "https://arxiv.org/pdf/1706.03762"

In [ ]:
# use PyPDF loaded to read and chunk the input document
loader = PyPDFLoader(document_uri)
docs = loader.load_and_split()

# Verify if pages are loaded correctly
docs[0]

### Create local Vector DB

In [ ]:
# Create an in-memory Vector DB using Chroma
vectordb = Chroma.from_documents(docs, embeddings)

In [ ]:
# Set Vector DB as retriever
retriever = vectordb.as_retriever()

### Create Q&A Chain

In [ ]:
# Create Q&A template for the Gemini Model
template = """You task is to answer questions related documents.
Use the following context to answer the question at the end.
{context}

Answers should be crisp.

Question: {question}
Helpful Answer:"""

# Create a prompt template for the q&a chain
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
)

# Pass prompts to q&a chain
chain_type_kwargs = {"prompt": PROMPT}

# Retriever arguments
retriever.search_kwargs = {"k": 3}

In [ ]:
# Setup a RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
# Test the chain with a sample question
query = "Who are the authors of paper on Attention is all you need"
result = qa({"query": query})
result

## Evaluation
### Create the evaluation set

In [ ]:
# Evaluation set with questions and ground_truth
questions = [
    "Who is the author of paper Attention is all you need",
    "What architecture is proposed in paper titled Attention is all you need?",
]
ground_truth = [
    "The authors of the paper 'Attention is all you need' are:\n\n* Ashish Vaswani\n* Noam Shazeer\n* Niki Parmar\n* Jakob Uszkoreit\n* Llion Jones\n* Aidan N. Gomez\n* Łukasz Kaiser\n* Illia Polosukhin",
    "Transformers architecture",
]

### Run the [Q&A chain](#create-qa-chain) on evaluation dataset 

In [ ]:
contexts = []
answers = []

# Generate contexts and answers for each question
for query in questions:
    result = qa({"query": query})
    contexts.append(
        [document.page_content for document in result.get("source_documents")]
    )
    answers.append(result.get("result"))

In [ ]:
# Convert into a dataset and prepare for consumtion by RAGAS API
data = {
    "question": questions,
    "contexts": contexts,
    "ground_truth": ground_truth,
    "answer": answers,
}

dataset = Dataset.from_dict(data)
dataset

In [ ]:
# Compile list of RAGAS Metrics
metrics = [
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
    answer_similarity,
]

## IMPORTANT : Gemini with RAGAS
> RAGAS is designed to work with OpenAI Models by default. We must set a few attributes to make it work with Gemini

In [ ]:
class RAGASVertexAIEmbeddings(VertexAIEmbeddings):
    """Wrapper for RAGAS"""

    async def embed_text(self, text: str) -> list[float]:
        """Embeds a text for semantics similarity"""
        return self.embed([text], 1, "SEMANTIC_SIMILARITY")[0]

In [ ]:
# Wrapper to make RAGAS work with Gemini and Vertex AI Embeddings Models
embeddings = RAGASVertexAIEmbeddings(model_name="textembedding-gecko@003")
ragas_llm = LangchainLLMWrapper(llm)

for m in metrics:
    # change LLM for metric
    m.__setattr__("llm", ragas_llm)

    # check if this metric needs embeddings
    if hasattr(m, "embeddings"):
        # if so change with Vertex AI Embeddings
        m.__setattr__("embeddings", embeddings)

### Run the RAGAS Evaluation

In [ ]:
# Run the evaluation on every row of the dataset
result_set = []
for i in range(len(dataset)):
    result = evaluate(
        dataset=Dataset.from_dict(dataset[i : i + 1]),
        metrics=metrics,
        raise_exceptions=False,
    )
    result_set.append(result.to_pandas())

In [ ]:
# View results in Pandas DataFrame
results_df = pd.concat(result_set)
results_df

# Conclusion

In this notebook, you learned:

1. RAGAS - Framework for evaluation .
2. Making RAGAS Work with Vertex AI Gemini API